In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models.detection as detection
import numpy as np
import os
import cv2
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from pycocotools.coco import COCO

In [2]:
class CocoDataset(Dataset):
    def __init__(self, root, annotation, transforms=None):
        self.root = root
        self.coco = COCO(annotation)
        self.image_ids = list(self.coco.imgs.keys())
        self.transforms = transforms

    def __getitem__(self, index):
        image_id = self.image_ids[index]
        annotation_ids = self.coco.getAnnIds(imgIds=image_id)
        annotations = self.coco.loadAnns(annotation_ids)
        
        # Görseli yükle
        image_path = os.path.join(self.root, self.coco.imgs[image_id]['file_name'])
        image = Image.open(image_path).convert("RGB")
        
        # Kutu ve etiketleri al
        boxes = []
        labels = []
        for ann in annotations:
            x, y, width, height = ann['bbox']
            boxes.append([x, y, x + width, y + height])
            labels.append(ann['category_id'])

        # PyTorch formatına dönüştür
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels
        }

        if self.transforms:
            image = self.transforms(image)

        return image, target

    def __len__(self):
        return len(self.image_ids)

In [13]:
import json
import random
import os
import shutil

# 📂 COCO JSON dosyasını oku
coco_path = r"C:\Users\524ha\Desktop\training_data\annotations.json"  # COCO formatındaki JSON dosyan
image_dir = r"C:\Users\524ha\Desktop\training_data\dataset" # Görsellerin bulunduğu klasör
output_dir = r"C:\Users\524ha\Desktop\training_data\output_dataset"  # Çıktıların kaydedileceği ana klasör

with open(coco_path, "r") as f:
    coco_data = json.load(f)

# 🎯 Train / Validation oranı
train_ratio = 0.8  # %80 Train, %20 Validation

# 📸 Tüm görüntüleri al ve karıştır
images = coco_data["images"]
random.shuffle(images)

# 🔀 Train - Validation ayır
train_size = int(len(images) * train_ratio)
train_images = images[:train_size]
val_images = images[train_size:]

# 📌 Image ID'leri al
train_ids = {img["id"] for img in train_images}
val_ids = {img["id"] for img in val_images}

# 🔍 Annotationları filtrele
train_annotations = [ann for ann in coco_data["annotations"] if ann["image_id"] in train_ids]
val_annotations = [ann for ann in coco_data["annotations"] if ann["image_id"] in val_ids]

# 🎯 Yeni JSON yapısını oluştur
train_coco = {
    "images": train_images,
    "annotations": train_annotations,
    "categories": coco_data["categories"]
}

val_coco = {
    "images": val_images,
    "annotations": val_annotations,
    "categories": coco_data["categories"]
}

# 📂 Train ve Validation klasörlerini oluştur
train_image_dir = os.path.join(output_dir, "train")
val_image_dir = os.path.join(output_dir, "val")
os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)

# 📌 Görselleri ilgili klasörlere taşı veya kopyala
for img in train_images:
    src_path = os.path.join(image_dir, img["file_name"])
    dst_path = os.path.join(train_image_dir, img["file_name"])
    if os.path.exists(src_path):
        shutil.copy(src_path, dst_path)  # Eğer taşımak istersen: shutil.move(src_path, dst_path)

for img in val_images:
    src_path = os.path.join(image_dir, img["file_name"])
    dst_path = os.path.join(val_image_dir, img["file_name"])
    if os.path.exists(src_path):
        shutil.copy(src_path, dst_path)

# 📂 Yeni JSON dosyalarını kaydet
with open(os.path.join(output_dir, "train.json"), "w") as f:
    json.dump(train_coco, f, indent=4)

with open(os.path.join(output_dir, "val.json"), "w") as f:
    json.dump(val_coco, f, indent=4)

print(f"✅ Train seti: {len(train_images)} görüntü")
print(f"✅ Validation seti: {len(val_images)} görüntü")
print(f"📂 Train görselleri kaydedildi: {train_image_dir}")
print(f"📂 Validation görselleri kaydedildi: {val_image_dir}")


✅ Train seti: 140 görüntü
✅ Validation seti: 36 görüntü
📂 Train görselleri kaydedildi: C:\Users\524ha\Desktop\training_data\output_dataset\train
📂 Validation görselleri kaydedildi: C:\Users\524ha\Desktop\training_data\output_dataset\val


In [4]:
# Görsel dönüşümleri
transform = transforms.Compose([
    transforms.ToTensor()
])

# Dataset & DataLoader
train_dataset = CocoDataset(root=r"C:\Users\524ha\Desktop\training_data\output_dataset\train", 
                            annotation=r"C:\Users\524ha\Desktop\training_data\output_dataset\train.json", 
                            transforms=transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))


loading annotations into memory...
Done (t=0.17s)
creating index...
index created!


In [5]:
def get_model(num_classes):
    model = detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    # Son sınıflandırma katmanını güncelle (num_classes: background + object classes)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    
    return model

# Kategori sayısı (arka plan + nesne sınıfları)
num_classes = 5  # Örneğin, sadece 1 nesne sınıfı varsa (background + class)
model = get_model(num_classes)


c:\Users\524ha\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\524ha\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# Optimizasyon ve Loss Fonksiyonu
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
num_epochs = 10  # Epoch sayısı

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for images, targets in train_loader:
        images = list(img for img in images)
        targets = [{k: v for k, v in t.items()} for t in targets]

        # Forward
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        # Backward
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        epoch_loss += losses.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Modeli kaydet
torch.save(model.state_dict(), "faster_rcnn_coco.pth")
print("Model kaydedildi!")


: 

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models.detection as detection
from torch.utils.data import DataLoader
from tqdm import tqdm
import time

# 📌 Cihaz ayarı (GPU varsa kullan, yoksa CPU'ya geç)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılan cihaz: {device}")

# 📌 Modeli yükle (num_classes = nesne sayısı + background)
num_classes = 5  # Örneğin, 1 sınıf ve background
model = detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
model.to(device)

# 📌 Veri seti (COCO formatında olduğu varsayılıyor)
# COCO veri yükleyicisini oluşturmalısın, burayı kendi datasetinle değiştir!
from torchvision.datasets import CocoDetection

transform = transforms.Compose([transforms.ToTensor()])
dataset = CocoDetection(root=r"C:\Users\524ha\Desktop\training_data\output_dataset\train", annFile=r"C:\Users\524ha\Desktop\training_data\output_dataset\train.json", transform=transform)
data_loader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

# 📌 Optimizer ve Loss
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0001)
num_epochs = 10  # Eğitilecek epoch sayısı

# 📌 Eğitim döngüsü
print("🚀 Eğitim başlıyor...")
for epoch in range(num_epochs):
    print(f"📅 Epoch {epoch+1}/{num_epochs}")

    epoch_start = time.time()
    model.train()
    
    running_loss = 0.0
    for batch_idx, (images, targets) in enumerate(tqdm(data_loader, desc=f"Epoch {epoch+1} ilerliyor")):
        try:
            images = [img.to(device) for img in images]  # Görselleri cihaza taşı
            
            # 🛠️ COCO formatını Faster R-CNN için uygun hale getiriyoruz
            new_targets = []
            for t in targets:
                boxes = torch.tensor([ann["bbox"] for ann in t], dtype=torch.float32)  # BBox'ları al
                labels = torch.tensor([ann["category_id"] for ann in t], dtype=torch.int64)  # Etiketleri al
                
                # Eğer bbox'lar (xmin, ymin, w, h) formatında ise (xmin, ymin, xmax, ymax) çevir
                boxes[:, 2] = boxes[:, 0] + boxes[:, 2]  # xmax = xmin + w
                boxes[:, 3] = boxes[:, 1] + boxes[:, 3]  # ymax = ymin + h
                
                new_targets.append({"boxes": boxes.to(device), "labels": labels.to(device)})

            optimizer.zero_grad()
            loss_dict = model(images, new_targets)  # ✅ Targets artık doğru formatta!
            loss = sum(loss for loss in loss_dict.values())

            loss.backward()
            optimizer.step()

            print(f"📌 Batch {batch_idx}, Loss: {loss.item():.4f}")

        except Exception as e:
            print(f"❌ Hata oluştu (Batch {batch_idx}): {str(e)}")

    epoch_end = time.time()
    avg_loss = running_loss / len(data_loader)
    print(f"✅ Epoch {epoch+1} tamamlandı. Ortalama Loss: {avg_loss:.4f} | Süre: {epoch_end - epoch_start:.2f} saniye")

print("🎉 Eğitim tamamlandı!")


Kullanılan cihaz: cuda
loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
🚀 Eğitim başlıyor...
📅 Epoch 1/10


Epoch 1 ilerliyor:   1%|▏         | 1/70 [00:02<02:36,  2.26s/it]

📌 Batch 0, Loss: 3.8689


Epoch 1 ilerliyor:   3%|▎         | 2/70 [00:02<01:23,  1.22s/it]

📌 Batch 1, Loss: 2.2175


...

📌 Batch 67, Loss: 0.1590


Epoch 10 ilerliyor:  99%|█████████▊| 69/70 [00:35<00:00,  1.96it/s]

📌 Batch 68, Loss: 0.2612


Epoch 10 ilerliyor: 100%|██████████| 70/70 [00:35<00:00,  1.96it/s]

📌 Batch 69, Loss: 0.1440
✅ Epoch 10 tamamlandı. Ortalama Loss: 0.0000 | Süre: 35.73 saniye
🎉 Eğitim tamamlandı!
